In [1]:
import os
import nltk
from pymorphy2 import MorphAnalyzer
pmm = MorphAnalyzer()
from nltk.tokenize import RegexpTokenizer
from pymorphy2.tokenizers import simple_word_tokenize
tokenizer = RegexpTokenizer(r'\w+')

import numpy as np
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('russian')
from nltk.parse import DependencyGraph
from collections import Counter
from nltk.collocations import *
import scipy
from scipy import stats

def normalize_text(text):
    text = tokenizer.tokenize(text.lower())
    lemmas = [pmm.parse(t)[0].normal_form for t in text]
    return ' '.join(lemmas)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\qwe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Парсим корпус

In [2]:
# !C:\Users\qwe\Desktop\cosyco\udpipe\udpipe-1.2.0-bin\bin-win64\udpipe --input horizontal --output conllu \
# --tokenize --tag --parse \
# C:\Users\qwe\Desktop\cosyco\udpipe\russian-syntagrus-ud-2.4-190531.udpipe < testset2.txt > text.conllu

In [3]:
trees = []
with open('text.conllu', 'r', encoding='utf-8') as f:
    parsed_sents = f.read().split('\n\n')
    for sent in parsed_sents:
        tree = [line for line in sent.split('\n') if line and line[0] != '#']
        trees.append('\n'.join(tree))
        

print(trees[2])

1	20	20	NUM	_	_	4	nummod	_	_
2	ноября	ноябрь	NOUN	_	Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing	1	flat	_	_
3	он	он	PRON	_	Case=Nom|Gender=Masc|Number=Sing|Person=3	4	nsubj	_	_
4	подал	подать	VERB	_	Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	0	root	_	_
5	в	в	ADP	_	_	7	case	_	_
6	арбитражный	арбитражный	ADJ	_	Animacy=Inan|Case=Acc|Degree=Pos|Gender=Masc|Number=Sing	7	amod	_	_
7	суд	суд	NOUN	_	Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing	4	obl	_	_
8	Москвы	Москва	PROPN	_	Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing	7	nmod	_	_
9	иск	иск	NOUN	_	Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing	4	obj	_	SpaceAfter=No
10	,	,	PUNCT	_	_	13	punct	_	_
11	в	в	ADP	_	_	12	case	_	_
12	котором	который	PRON	_	Case=Loc	13	obl	_	_
13	просит	просить	VERB	_	Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act	9	acl:relcl	_	_
14	признать	признать	VERB	_	Aspect=Perf|VerbForm=Inf|Voice=Act	13	xcomp	_	_
15	недействительным	недействительный	ADJ	_	Case=Ins|

#### Строим частотный словарь глаголов

In [4]:
verb_freq = Counter()
for one_tree in trees:
    try:
        g = DependencyGraph(one_tree, top_relation_label='root')
        for n in g.nodes:
            if g.nodes[n]['ctag'] == 'VERB':
                verb_freq[g.nodes[n]['lemma']] += 1
    except:
        pass
verb_freq_50 = [item[0] for item in verb_freq.most_common() if item[1] >= 50]
print(verb_freq_50)

['обвинить', 'мочь', 'признать', 'подать', 'заявить', 'сообщить', 'стать', 'удовлетворить', 'требовать', 'вынести', 'считать', 'принять', 'просить', 'обратиться', 'говорить', 'быть', 'получить', 'приговорить', 'обвинять', 'отказаться', 'находиться', 'являться', 'направить', 'обжаловать', 'рассматривать', 'сказать', 'напомнить', 'предъявить', 'иметь', 'подтвердить', 'рассмотреть', 'пытаться', 'решить', 'отменить', 'арестовывать', 'делать', 'начаться', 'объявить', 'утверждать', 'дать', 'отклонить', 'передать', 'нет', 'оспорить', 'назначить', 'провести', 'согласиться', 'взыскать', 'оказаться', 'удаться', 'оставить', 'отказать', 'выплатить', 'обязать', 'пояснить', 'использовать', 'доказать', 'связать', 'счесть', 'запретить', 'рассказать', 'постановить', 'принадлежать', 'оспаривать', 'пройти']


#### Собираем все биграммы (глагол + прямое дополнение)

-> биграммы в виде лемм

In [5]:
#bigrams = Counter()
bigrams = []
for one_tree in trees:
    try:
        g = DependencyGraph(one_tree, top_relation_label='root')
        for item in g.triples():
            if item[1] == 'obj' and item[2][1] == 'NOUN':
                lemma = normalize_text(item[0][0])
                if lemma in verb_freq_50:
                    coll = tuple([normalize_text(item[0][0]), 
                                  normalize_text(item[2][0])])
                    bigrams.append(coll)
                    # bigrams[tuple([item[0][0], item[2][0]])] += 1
    except:
        pass

In [6]:
bigrams[:20]

[('подать', 'иск'),
 ('признать', 'договор'),
 ('рассмотреть', 'вопрос'),
 ('обвинить', 'руководство'),
 ('получить', 'срок'),
 ('обвинить', 'олигарх'),
 ('направить', 'ведомство'),
 ('просить', 'суд'),
 ('рассматривать', 'ходатайство'),
 ('отказать', 'жалоба'),
 ('иметь', 'вид'),
 ('оставить', 'заключение'),
 ('дать', 'согласие'),
 ('признать', 'сделка'),
 ('обвинить', 'господин'),
 ('удовлетворить', 'жалоба'),
 ('вынести', 'постановление'),
 ('получить', 'доля'),
 ('вынести', 'приговор'),
 ('признать', 'сделка')]

#### Оцениваем по метрикам log-likelihood, dice, PMI

In [7]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(bigrams)
finder.apply_word_filter(lambda w: len(w) < 3 or w.lower() in nltk.corpus.stopwords.words('russian'))

In [8]:
loglike_scores = {i[0]:i[1] for i in finder.score_ngrams(bigram_measures.likelihood_ratio)}
pmi_scores = {i[0]:i[1] for i in finder.score_ngrams(bigram_measures.pmi)}
dice_scores = {i[0]:i[1] for i in finder.score_ngrams(bigram_measures.dice)}

In [9]:
pmi_100 = finder.nbest(bigram_measures.pmi, 100)
loglike_100 = finder.nbest(bigram_measures.likelihood_ratio, 100)
dice_100 =  finder.nbest(bigram_measures.dice, 100)

Находим пересечение трех метрик

In [10]:
three_metrics = set(pmi_100) & set(loglike_100) & set(dice_100)
three_metrics

{('взыскать', 'задолженность'),
 ('взыскать', 'неустойка'),
 ('выплатить', 'иркутянин'),
 ('делать', 'вид'),
 ('делать', 'вывод'),
 ('доказать', 'невиновность'),
 ('доказать', 'незаконность'),
 ('запретить', 'деятельность'),
 ('заявить', 'отвод'),
 ('назначить', 'наказание'),
 ('напомнить', 'основание'),
 ('объявить', 'голодовка'),
 ('объявить', 'перерыв'),
 ('объявить', 'предприниматель'),
 ('обязать', 'прокуратура'),
 ('отказать', 'оао'),
 ('пройти', 'прение'),
 ('решить', 'проблема')}

Словарь глагольной сочетаемости

* извлекае все словосочетания (из последней колонки) в том порядке, в котором они даны
* лемматизируем



In [11]:
with open('verb_coll.txt', 'r', encoding='utf-8') as verb_coll:
    verb_colls = verb_coll.read().split('\n')

verb_colls = [t.split('\t')[-1] for t in verb_colls]
verb_colls = [tuple(normalize_text(t).split()) for t in verb_colls]

#### Находим Золотой стандарт

In [12]:
Gold = set(verb_colls) & set(pmi_100) & set(loglike_100) & set(dice_100)
Gold

{('делать', 'вид'),
 ('делать', 'вывод'),
 ('объявить', 'перерыв'),
 ('решить', 'проблема')}

In [13]:
# Что осталось за пределами ЗС?
three_metrics - Gold

{('взыскать', 'задолженность'),
 ('взыскать', 'неустойка'),
 ('выплатить', 'иркутянин'),
 ('доказать', 'невиновность'),
 ('доказать', 'незаконность'),
 ('запретить', 'деятельность'),
 ('заявить', 'отвод'),
 ('назначить', 'наказание'),
 ('напомнить', 'основание'),
 ('объявить', 'голодовка'),
 ('объявить', 'предприниматель'),
 ('обязать', 'прокуратура'),
 ('отказать', 'оао'),
 ('пройти', 'прение')}

К ЗС можно было бы добавить следующие коллокации:

* Объявить голодовку
* заявить отвод
* взыскать неустойку

Критерием выделения коллокаций можно считать их фразеологичность и ограниченную сочетаемость составляющих коллокацию слов. Т.е чтобы проверить, является ли словосочетание коллокацией, нужно попробовать заменить слова

Попробуем: 

* *сказать/проговорить/пообещать/заявить/прокричать голодовку
* *объявить/сказать/проговорить отвод
* ?собрать/взять/забрать неустойку (сказать что-то типа потребовать неустойку, наверное, можно, но взыскать неустойку звучит привычнее)


In [14]:
Gold |= set([('взыскать', 'неустойка'), ('заявить', 'отвод'), ('объявить', 'голодовка')])
Gold

{('взыскать', 'неустойка'),
 ('делать', 'вид'),
 ('делать', 'вывод'),
 ('заявить', 'отвод'),
 ('объявить', 'голодовка'),
 ('объявить', 'перерыв'),
 ('решить', 'проблема')}

#### Считаем ранговую корреляцию

In [15]:
data_loglike = dict.fromkeys(Gold)
for item in Gold:
    data_loglike[item] = loglike_scores[item]
data_loglike

{('заявить', 'отвод'): 46.409863151989065,
 ('делать', 'вид'): 21.540900542578598,
 ('делать', 'вывод'): 31.787851302988763,
 ('решить', 'проблема'): 39.537197250991326,
 ('взыскать', 'неустойка'): 22.57407317987232,
 ('объявить', 'перерыв'): 23.540838183907574,
 ('объявить', 'голодовка'): 35.60186724594653}

In [16]:
data_pmi = dict.fromkeys(Gold)
for item in Gold:
    data_pmi[item] = pmi_scores[item]
data_pmi

{('заявить', 'отвод'): 8.141012309527074,
 ('делать', 'вид'): 8.363404730863522,
 ('делать', 'вывод'): 8.211401637418472,
 ('решить', 'проблема'): 9.141012309527074,
 ('взыскать', 'неустойка'): 8.04147663597616,
 ('объявить', 'перерыв'): 8.363404730863522,
 ('объявить', 'голодовка'): 8.363404730863522}

In [17]:
data_dice = dict.fromkeys(Gold)
for item in Gold:
    data_dice[item] = dice_scores[item]
data_dice

{('заявить', 'отвод'): 0.4444444444444444,
 ('делать', 'вид'): 0.36363636363636365,
 ('делать', 'вывод'): 0.46153846153846156,
 ('решить', 'проблема'): 0.6,
 ('взыскать', 'неустойка'): 0.23529411764705882,
 ('объявить', 'перерыв'): 0.2857142857142857,
 ('объявить', 'голодовка'): 0.4}

* Если считать, что коллокации в золотом стандарте - эталон, и приписать им всем оценку, близкую к 1.0, то корреляции не будет
* Поэтому для каждой коллокации я попробовала взять ее усредненную оценку для всех трех метрик

In [18]:
data0 = [0.9] * 7
stats_m = [
    list(data_loglike.values()),
    list(data_pmi.values()),
    list(data_dice.values())
]

In [19]:
for i in stats_m:
    print(stats.spearmanr(data0, i))

SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)


c:\users\qwe\appdata\local\programs\python\python37-32\lib\site-packages\numpy\lib\function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
c:\users\qwe\appdata\local\programs\python\python37-32\lib\site-packages\numpy\lib\function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
c:\users\qwe\appdata\local\programs\python\python37-32\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
c:\users\qwe\appdata\local\programs\python\python37-32\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
c:\users\qwe\appdata\local\programs\python\python37-32\lib\site-packages\scipy\stats\_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


*берем среднее между тремя метриками*

In [20]:
data1 = dict.fromkeys(Gold)
for colloc in data1:
    data1[colloc] = np.mean([pmi_scores[colloc], loglike_scores[colloc], dice_scores[colloc]])
data1

{('заявить', 'отвод'): 18.33177330198686,
 ('делать', 'вид'): 10.089313879026161,
 ('делать', 'вывод'): 13.486930467315231,
 ('решить', 'проблема'): 16.42606985350613,
 ('взыскать', 'неустойка'): 10.283614644498513,
 ('объявить', 'перерыв'): 10.729985733495127,
 ('объявить', 'голодовка'): 14.788423992270017}

In [21]:
for i in stats_m:
    print(stats.spearmanr(list(data1.values()), i))

SpearmanrResult(correlation=1.0, pvalue=0.0)
SpearmanrResult(correlation=0.11118739749916519, pvalue=0.8124070154613051)
SpearmanrResult(correlation=0.7142857142857144, pvalue=0.07134356146753759)


Попробуем сами отранжировать коллокации

In [22]:
data3 = [6, 3, 2, 4, 7, 5, 1]
for i in stats_m:
    print(stats.spearmanr(list(data3), i))

SpearmanrResult(correlation=-0.03571428571428572, pvalue=0.9394082054712856)
SpearmanrResult(correlation=-0.5188745216627708, pvalue=0.23275391866676248)
SpearmanrResult(correlation=-0.42857142857142866, pvalue=0.337368311085824)


Наибольшее значение корреляции у pmi, у других метрик значение корреляции сильно хуже

PMI принимает во внимание именно то, насколько часто слова встречаются вместе, при учете их вероятностей по-отдельности

Возможно, коллокации - это редкие слова по-отдельности, которые встречаются зачастую вместе, поэтому по-отдельности они не так значимы, как их сочетание

Попробуем посчитать рандомную корреляцию

In [23]:
np.random.seed(42)
data2 =  np.random.random_sample((7,))
for i in stats_m:
    print(stats.spearmanr(data2, i))
data2

SpearmanrResult(correlation=-0.21428571428571433, pvalue=0.6445115810207203)
SpearmanrResult(correlation=0.07412493166611013, pvalue=0.8745070324258755)
SpearmanrResult(correlation=0.39285714285714296, pvalue=0.38331687042697266)


array([0.37454012, 0.95071431, 0.73199394, 0.59865848, 0.15601864,
       0.15599452, 0.05808361])

In [24]:
set(pmi_100[:30]) & Gold, set(pmi_100[:50]) & Gold

({('решить', 'проблема')},
 {('взыскать', 'неустойка'),
  ('делать', 'вид'),
  ('делать', 'вывод'),
  ('заявить', 'отвод'),
  ('объявить', 'голодовка'),
  ('объявить', 'перерыв'),
  ('решить', 'проблема')})

In [25]:
set(loglike_100[:30]) & Gold, set(loglike_100[:50]) & Gold

({('заявить', 'отвод')},
 {('заявить', 'отвод'), ('объявить', 'голодовка'), ('решить', 'проблема')})

In [26]:
set(dice_100[:30]) & Gold, set(dice_100[:50]) & Gold

({('делать', 'вывод'),
  ('заявить', 'отвод'),
  ('объявить', 'голодовка'),
  ('решить', 'проблема')},
 {('делать', 'вид'),
  ('делать', 'вывод'),
  ('заявить', 'отвод'),
  ('объявить', 'голодовка'),
  ('объявить', 'перерыв'),
  ('решить', 'проблема')})

In [27]:
loglike_scores

{('подать', 'иск'): 372.0663717307137,
 ('принять', 'решение'): 199.97206482692806,
 ('удовлетворить', 'иск'): 174.00267243011012,
 ('вынести', 'приговор'): 173.4241895896191,
 ('просить', 'суд'): 170.67389434450098,
 ('обжаловать', 'решение'): 163.60425211415554,
 ('вынести', 'решение'): 121.1907748737116,
 ('удовлетворить', 'ходатайство'): 120.99306215033499,
 ('отклонить', 'жалоба'): 99.42317024987742,
 ('предъявить', 'обвинение'): 97.14767931917764,
 ('иметь', 'право'): 88.75585398247395,
 ('передать', 'дело'): 84.76965413151152,
 ('пройти', 'прение'): 83.28440956930956,
 ('признать', 'договор'): 75.14236307161813,
 ('рассматривать', 'дело'): 74.1032224770862,
 ('запретить', 'деятельность'): 72.43824159291282,
 ('обвинить', 'компания'): 68.8953332048377,
 ('выплатить', 'компенсация'): 68.23239094981196,
 ('рассмотреть', 'жалоба'): 67.0903917040022,
 ('обвинить', 'президент'): 56.051736914811514,
 ('подать', 'апелляция'): 54.62499600670342,
 ('рассмотреть', 'вопрос'): 54.46043749726

In [28]:
pmi_scores

{('рассказать', 'замгендиректор'): 11.948367231584678,
 ('являться', 'долг'): 11.948367231584678,
 ('арестовывать', 'счёт'): 10.948367231584678,
 ('отказаться', 'падение'): 10.948367231584678,
 ('стать', 'сальмонелла'): 10.948367231584678,
 ('мочь', 'выход'): 10.363404730863522,
 ('обратиться', 'ответ'): 10.363404730863522,
 ('обратиться', 'родные'): 10.363404730863522,
 ('пояснить', 'причина'): 10.363404730863522,
 ('принадлежать', 'танкер'): 10.363404730863522,
 ('напомнить', 'основание'): 9.948367231584678,
 ('отказаться', 'окружение'): 9.948367231584678,
 ('постановить', 'день'): 9.948367231584678,
 ('постановить', 'ущерб'): 9.948367231584678,
 ('сообщить', 'сведение'): 9.948367231584678,
 ('начаться', 'экс глава'): 9.626439136697316,
 ('мочь', 'судьба'): 9.363404730863522,
 ('обратиться', 'польза'): 9.363404730863522,
 ('пояснить', 'адвокат'): 9.363404730863522,
 ('решить', 'проблема'): 9.141012309527074,
 ('требовать', 'отделение'): 9.141012309527074,
 ('требовать', 'порядок'): 9

In [29]:
dice_scores

{('рассказать', 'замгендиректор'): 1.0,
 ('являться', 'долг'): 1.0,
 ('арестовывать', 'счёт'): 0.6666666666666666,
 ('напомнить', 'основание'): 0.6666666666666666,
 ('отказаться', 'падение'): 0.6666666666666666,
 ('пройти', 'прение'): 0.6666666666666666,
 ('стать', 'сальмонелла'): 0.6666666666666666,
 ('решить', 'проблема'): 0.6,
 ('запретить', 'деятельность'): 0.5833333333333334,
 ('просить', 'суд'): 0.5555555555555556,
 ('подать', 'иск'): 0.5073746312684366,
 ('мочь', 'выход'): 0.5,
 ('обратиться', 'ответ'): 0.5,
 ('обратиться', 'родные'): 0.5,
 ('отказаться', 'окружение'): 0.5,
 ('пояснить', 'причина'): 0.5,
 ('принадлежать', 'танкер'): 0.5,
 ('делать', 'вывод'): 0.46153846153846156,
 ('назначить', 'наказание'): 0.46153846153846156,
 ('выплатить', 'компенсация'): 0.45714285714285713,
 ('заявить', 'отвод'): 0.4444444444444444,
 ('предъявить', 'обвинение'): 0.43333333333333335,
 ('вынести', 'приговор'): 0.4,
 ('мочь', 'судьба'): 0.4,
 ('обратиться', 'польза'): 0.4,
 ('объявить', 'голо

### Анализ

* Оценка корреляции показала, что ближе всего к ЗС оказалась метрика loglikelihood
* Меньше всего корреляция для PMI, на втором месте расположилась dice
* Если мы сами оценим, насколько высокую оценку дают метрики для коллокаций, вошедших в ЗС, то увидим, что точнее всего оказывается dice: в первых 30 коллокаций встретилось больше сочетаний,вошедших в ЗС, чем для остальных метрик
* Кажется, что для pmi важна еще и частотность отдельного слова в коллокации, если посмотреть на выдачу, то можно заметить, будто распределение коллокаций как будто сгруппировано по отдельным словам - вершинам коллокаций
* Loglikelihood скорее в данном случае выбрал самые частотные сочетания слов, самые частотные биграммы. Вопрос: частотное словосочетание == коллокация?
* Dice учитывает совместные сочетания слов, и чем частотнее слова встречаются вместе, чем меньше частотность отдельных слов, тем лучше. Однако ошибочны просто редкие сочетания, когда коллокация встретилась дишь один раз, и слова в ней встретились только в этой коллокации, оказываются в топе, что приводит к ошибочному выделению коллокации (('рассказать', 'замгендиректор'), ('являться', 'долг'))